In [1]:
import Control.Arrow

type Algebra f a = f a -> a
type Coalgebra f a = a -> f a

newtype Fix f = Fix { unFix :: f (Fix f) }

cata :: Functor f => Algebra f a -> Fix f -> a
cata alg = alg . fmap(cata alg) . unFix

ana :: Functor f => Coalgebra f a -> a -> Fix f
ana coalg = Fix . fmap (ana coalg) . coalg

-- we can abandon ana and cata by composing them to yield hylo
hyloInefficient :: Functor f => Algebra f a -> Coalgebra f b -> b -> a
hyloInefficient alg coalg = ana coalg >>> cata alg

hylo :: Functor f => Algebra f a -> Coalgebra f b -> b -> a
hylo f g = f . fmap (hylo f g) . g

In [2]:
{-# LANGUAGE DeriveFunctor #-}

--List functor carrying type t
data ListF t a = Cons t a | Empty
    deriving (Functor,Show)

--recursive list type
data ListR t = ConsR t (ListR t) | EmptyR
    deriving (Functor,Show)

In [3]:
toList :: Algebra (ListF t) [t]
toList Empty = []
toList (Cons x y) = x : y

fromList :: Coalgebra (ListF t) [t]
fromList [] = Empty
fromList (x : y) = Cons x y

hylo toList fromList [1,2,3]

[1,2,3]

In [4]:
toListR :: Algebra (ListF t) (ListR t)
toListR Empty = EmptyR
toListR (Cons x y) = ConsR x y

fromListR :: Coalgebra (ListF t) (ListR t)
fromListR EmptyR = Empty
fromListR (ConsR x y) = Cons x y

hylo toListR fromList [1,2,3]

ConsR 1 (ConsR 2 (ConsR 3 EmptyR))

In [5]:
sumAlg :: Algebra (ListF Int) Int
sumAlg Empty = 0
sumAlg (Cons x y) = x + y

hylo sumAlg fromList [1,2,3]

6

In [6]:
unfoldDescend :: Coalgebra (ListF Int) Int
unfoldDescend n
    | n == 0 = Empty
    | otherwise = Cons n (n-1)
    
hylo toList unfoldDescend 5

[5,4,3,2,1]

In [7]:
write :: Algebra (ListF Int) String
write Empty = "Empty"
write (Cons n s) = "Cons " ++ show n ++ " (a . (" ++ s ++ ")"

hylo write fromList [1,2,3]

"Cons 1 (a . (Cons 2 (a . (Cons 3 (a . (Empty)))"

In [8]:
foldAlg :: (t -> t -> t) -> t -> Algebra (ListF t) t
foldAlg f x0 Empty = x0
foldAlg f x0 (Cons y z) = f y z

foldList :: (t -> t -> t) -> t -> [t] -> t
foldList f x0 = hylo (foldAlg f x0) fromList

foldList (+) 1 [1,2,3]

7

In [9]:
foldAlgCPS :: (b -> a -> b) -> Algebra (ListF a) (b -> b)
foldAlgCPS f Empty = id
foldAlgCPS f (Cons n g) = \m -> g $ f m n

hylo (foldAlgCPS (+)) fromList [1,2,3] 4

10